# <b><font color='#009e74'>Variant/Residue effect class prediction via mutational scanning.</font></b>
Colaboratory implementation of : **Cagiada, et al.,** [Discovering functionally important sites in proteins
](https://www.biorxiv.org/content/10.1101/2022.07.14.500015v1.full). Source code is available on the project [Github](https://github.com/KULL-Centre/papers/tree/main/2022/functional-sites-cagiada-et-al) page.

Prediction of protein variants and residues on stability and function via evaluation of mutation effects. The program requires as input:
- **thermodynamic stability evaluation** ($\Delta \Delta G$ in kcal/mol) generated via e.g. Rosetta or [RaSP](https://github.com/KULL-Centre/papers/tree/main/2022/ML-ddG-Blaabjerg-et-al);
- the query protein **structure** (experimental crystal structure or AF2 prediction, in PDB format); 
- **conservation scores** ($\Delta \Delta E$) generated using [GEMME](https://academic.oup.com/mbe/article/36/11/2604/5548199?login=true), which can be generated in the official [webserver](http://www.lcqb.upmc.fr/GEMME/submit.html). 
The output files generated will include input features per variant, prediction per variants and prediction per positions.

##  <b><font color='#009e74'> Reminders and Important informations:</font></b>
- This notebook  <b><font color='#d55c00'>can </font></b> be run without using a Colab GPU session (to do: go to page menu: `Runtime`->  `Change runtime type` -> select `CPU` and confirm)
- Cells named as  <b><font color='#f0e422'>PRELIMINARY OPERATIONS </font></b> have to be run <b><font color='#d55c00'>ONCE only at the start</font></b>  and  skipped for new predictions.
- <b><font color='#d55c00'>ONE</font></b> single protein at the time can be processed by the pipeline, but re-running the **Load sequence** cell allows to run multiple proteins in series. 
- <b><font color='#d55c00'>Each protein</font></b> requires a series of input from outside and specific formatted file. Please check the file format on the [Github](https://github.com/KULL-Centre/papers/tree/main/2022/functional-sites-cagiada-et-al) repository.
-  <b><font color='#56b4e9'>Query sequence</font></b> is the reference sequence for the protein during the analysis. Scores for GEMME, $\Delta \Delta G$ and structure features should be evaluated on the query sequence. If this is not possible, the query sequence and the specific score sequence will be align and only matching residues will be consider for the analysis.

****

In [ ]:
#@title <b><font color='#f0e422'>PRELIMINARY OPERATIONS:</font> Install dependencies
#@markdown Run the cell to install all the extra necessaries packages, including:
#@markdown - catboost (0.26.1 - version used in the manuscript)
#@markdown - MDtraj
#@markdown - Biopython

%%bash -s

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  pip install -q catboost==0.26.1 ## same version used in the paper
  pip install -q mdtraj
  pip install -q Biopython

fi
wget -cq https://github.com/KULL-Centre/papers/raw/main/2022/functional-sites-cagiada-et-al/catboost_model/cat_param_trained.zip

unzip cat_param_trained.zip 

rm cat_param_trained.zip


In [2]:
#@title <b><font color='#f0e422'>PRELIMINARY OPERATIONS:</font> Load pipeline functions
#@markdown Run the cell to load the required functions

import numpy as np
import pandas as pd
import mdtraj as md
import matplotlib.pyplot as plt
import matplotlib as mpl
import os,shutil
from google.colab import files
from Bio import Align

## dictionaries
alphabetAA_L_D={'-':0,'_' :0,'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,'M':11,'N':12,'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20}
alphabetAA_D_L={v: k for k, v in alphabetAA_L_D.items()}

AA_to_hydrophobicity_scores={'A':44,'C':50,'D':-37,'E':-12,'F':96,'G':0,'H':-16,'I':100,'K':-30,'L':99,'M':74,'N':-35,'P':-46,'Q':-14,'R':-20,'S':-6,'T':13,'V':78,'W':90,'Y':57}

alphabet_class_variant_d_l={0:'WT-like',1:'SBI',2:'Total-loss'}
alphabet_class_residue_d_l={0:'WT-like',1:'Functional site',2:'Total-loss',5:'Not assgined'}

aa_order_alphabetical = pd.Series(["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M",
           "N", "P", "Q", "R", "S", "T", "V", "W", "Y"])

alphabetAA_L_D_reordered={0:'R',1:'H',2:'K',3:'D',4:'E',5:'S',6:'T',7:'N',8:'Q',9:'C',10:'G',11:'P',12:'A',13:'V',14:'I',15:'L',16:'M',17:'F',18:'Y',19:'W'}
alphabetAA_D_D_reordered={14:0,6:1,8:2,2:3,3:4,15:5,16:6,11:7,13:8,1:9,5:10,12:11,0:12,17:13,7:14,9:15,10:16,4:17,19:18,18:19}

## functions

def remove_WT_score(score,WT_seq):
    for i in range(len(WT_seq)):
        score[i,alphabetAA_L_D[WT_seq[i]]-1]=np.nan
    return score

def load_data(data,wt_seq,column_score=1,print_sequences=False):
    df=pd.read_csv(data, delim_whitespace=True, comment='#')
    mutation_load=np.array(df.iloc[:,0])
    score_load=np.array(df.iloc[:,column_score])

    pos=1
    data_wt=''
    for i in range(len(mutation_load)):
        if i==0:
            pos=int(mutation_load[i][1:-1])
            if pos ==1:
                data_wt=data_wt+str(mutation_load[i][0])
            else:
                data_wt=data_wt+'-'*(pos-1)+str(mutation_load[i][0])
        else:
            #print(pos,int(mutation_load[i][1:-1]))
            if pos!= int(mutation_load[i][1:-1]):
                diff= int(mutation_load[i][1:-1]) - pos
                if diff ==1:
                    data_wt=data_wt+str(mutation_load[i][0])
                else:
                    data_wt=data_wt+'-'*(diff-1)+str(mutation_load[i][0])
                pos=int(mutation_load[i][1:-1])

    if wt_seq is None:
        wt_seq=data_wt
    aligner = Align.PairwiseAligner()
    alignments = aligner.align(wt_seq, data_wt)
    if print_sequences:
      print('query = wt_sequence / target: file_sequence')
      print(alignments[0])
    aligned_seqs=alignments[0].aligned

    scores=np.empty((len(data_wt),20),dtype=float)
    scores[:]=np.nan
    
    for i in range(len(mutation_load)):
        if mutation_load[i][len(mutation_load[i])-1]!= '=':
            scores[int(mutation_load[i][1:len(mutation_load[i])-1])-1, alphabetAA_L_D[mutation_load[i][len(mutation_load[i])-1]]-1]= float(score_load[i])
    
    scores_aligned=np.empty((len(wt_seq),20),dtype=float)
    scores_aligned[:]=np.nan
    
    for r in range(len(aligned_seqs[0])):
        scores_aligned[aligned_seqs[0,r][0]:aligned_seqs[0,r][1],:]=scores[aligned_seqs[1,r][0]:aligned_seqs[1,r][1],:]
    
    
    return scores_aligned

def WCN(pdb_loc,scheme_e,WT,chain=0,print_sequences=False):
    r0=7.0
    pdb=md.load(pdb_loc)
    topology=pdb.topology
    
    chainA=topology.select(f'chainid {chain} and protein')
    
    pdb_seq=topology.to_fasta(chain)
    
    pdb_chain0=pdb.atom_slice(chainA)
    pdb_dist,pdb_rp=md.compute_contacts(pdb_chain0,scheme=scheme_e,periodic=False)
  
    cm= md.geometry.squareform(pdb_dist,pdb_rp)[0]
    wcn=np.zeros((len(WT)),dtype=float)
    
    cm_adj=np.empty((len(WT),len(WT)),dtype=float)
    cm_adj[:]=np.nan
    
    chainA_top=pdb_chain0.topology
    aligner = Align.PairwiseAligner()
    alignments = aligner.align(WT, pdb_seq)
    
    aligned_seqs=alignments[0].aligned
    
    if print_sequences:
      print('query = wt_sequence / target: file_sequence')
      print(alignments[0])
    for r1 in range(len(aligned_seqs[0])):
      for r2 in range(len(aligned_seqs[0])):
        cm_adj[aligned_seqs[0,r1][0]:aligned_seqs[0,r1][1],aligned_seqs[0,r2][0]:aligned_seqs[0,r2][1]]= cm[aligned_seqs[1,r1][0]:aligned_seqs[1,r1][1],aligned_seqs[1,r2][0]:aligned_seqs[1,r2][1]]

    for i in range(len(WT)):
      nan_flag=True
      for j in range(len(WT)):
        if np.isnan(cm_adj[i,j])!=True and cm_adj[i,j]!=0.0:
          nan_flag=False
          wcn[i]+=(1-(cm_adj[i,j]*10/r0)**6)/(1-(cm_adj[i,j]*10/r0)**12)
      if nan_flag==True:
        wcn[i]=np.nan
    return wcn

def neighbor_scores(score,ext_range):
  score_neighborhood=np.zeros(len(score),dtype=float)
  for i in range(len(score)):
      if np.isnan(score[i])!=True:
          count_nan=0
          if i==0:
              for j in range(1,ext_range+1):
                  if np.isnan(score[j])==False:
                      score_neighborhood[i]+=score[j]
                  else:
                      count_nan+=1
              if count_nan!=ext_range:    
                  score_neighborhood[i]/=(ext_range)
              else:
                  score_neighborhood[i]=np.nan

          elif i==(len(score)-1):
              for j in range(len(score)-1-ext_range,len(score)-1):
                  if np.isnan(score[j])==False:
                      score_neighborhood[i]+=score[j]
                  else:
                      count_nan+=1
              if count_nan!=ext_range: 
                  score_neighborhood[i]/=ext_range
              else:
                  score_neighborhood[i]=np.nan                
          elif i<ext_range:
              for j in range(0,i+ext_range+1):
                  if j!=i:
                      if np.isnan(score[j])==False:
                          score_neighborhood[i]+=score[j]
                      else:
                          count_nan+=1
              if count_nan!=(i+ext_range):    
                  score_neighborhood[i]/=(i+ext_range)
              else:
                  score_neighborhood[i]=np.nan                        

          elif i>(len(score)-1-ext_range):
              for j in range(i-ext_range,len(score)):
                  if j!=i:
                      if np.isnan(score[j])==False:
                          score_neighborhood[i]+=score[j]
                      else:
                          count_nan+=1
              if count_nan!=(len(score)-i+ext_range):                     
                  score_neighborhood[i]/=(len(score)-i+ext_range)
              else:
                  score_neighborhood[i]=np.nan  
          else:
              for j in range(i-ext_range,i+ext_range+1):
                  if j!=i:
                      if np.isnan(score[j])==False:
                          score_neighborhood[i]+=score[j]
                      else:
                          count_nan+=1
              if count_nan!=(2*ext_range):  
                  score_neighborhood[i]/=(2*ext_range)
              else:
                  score_neighborhood[i]=np.nan             
      else:
          score_neighborhood[i]=np.nan
  return score_neighborhood

def normalize_cutoff(scores,lowcut,highcut):
  normalized_scores=np.copy(scores)
  for i in range(scores.shape[0]):
      for j in range(scores.shape[1]):
          if scores[i,j] < lowcut:
              normalized_scores[i,j]=lowcut
          elif scores[i,j] > highcut:
              normalized_scores[i,j]=highcut
          else:
              normalized_scores[i,j]=scores[i,j]
  return normalized_scores

def position_mean(score):
  score_mean=np.zeros(score.shape[0],dtype=float)
  for i in range(score.shape[0]):
      count=0
      flag_nan=True
      for j in range(score.shape[1]):
          if np.isnan(score[i,j])==False:
              flag_nan=False
              score_mean[i]+=score[i,j]
              count+=1
          else:
              pass
      if flag_nan==True:
          score_mean[i]=np.nan
      score_mean[i]/=count
      
  return score_mean

def features_validation(list_features_x,WT):
    
  X=[]
  mapping_pos=[] 
  
  for i in range(len(WT)):
      for j in range(20):
          if j!=(alphabetAA_L_D[WT[i]]-1):
            
              temp_x=[]
              temp_y=[]
              cond=True

              for elem in list_features_x:
                  if elem.ndim==1:
                      if np.isnan(elem[i])==True:
                          cond=False
                  else:
                      if np.isnan(elem[i,j])==True:
                          cond=False    
          

              if cond==True:
                  
                  for elem in list_features_x:
                      if elem.ndim==1:
                          temp_x.append(elem[i])
                      else:
                          temp_x.append(elem[i,j])
              if len(temp_x)>0:
                  X.append(temp_x)
                  mapping_pos.append([i,j])
      
  return np.array(X),mapping_pos

def retrieve_residue_label_pred(prediction,variant_map,WT,percentage_threshold):
  scores=np.empty((len(WT),20),dtype=float)
  scores[:]=np.nan
  
  i=0
  for cord in variant_map:
      scores[cord[0],cord[1]]=prediction[i]
      i+=1
  
  count_pos=np.zeros((len(WT),4),dtype=float)
  
  pseudomode_value_class=np.empty(len(WT),dtype=float)
  pseudomode_value_class[:]=np.nan
  percentage=np.zeros((len(WT),2),dtype=float)
      
  for i in range(len(pseudomode_value_class)):
      count=0
      for j in range(scores.shape[1]):
          if np.isnan(scores[i,j])!=True:
              count+=1
      if count>0:
          for j in range(scores.shape[1]):
                  if np.isnan(scores[i,j])!=True:
                      count_pos[i,int(scores[i,j])]+=1
          count_pos=count_pos/count
          if np.any(count_pos[i,:]>=0.50):
              pseudomode_value_class[i]=np.argmax(count_pos[i,:])
              percentage[i,0]=count_pos[i,1]
              #print(i, count_pos[i,:],np.max(count_pos[i,:]),np.argmax(count_pos[i,:]),pseudomode_value_class[i])


          else:                
              pseudomode_value_class[i]=5
              percentage[i,0]=count_pos[i,1]
              ## 5 indicates mixed signal with no predominat mutation class
          
          percentage[i,1]=count
    
  return pseudomode_value_class

def count_class_variant_position(variant_matrix,pred_class):
  variant_class_count=[]
  for i in range(variant_matrix.shape[0]):
    variant_class_count.append(np.count_nonzero((variant_matrix[i,:]==pred_class)))
  return np.array(variant_class_count)


def generate_empty_prism_df(seq):
    variant_list = []
    for wt, pos in zip(seq, range(len(seq))):
        for mut in aa_order_alphabetical:
            variant = str(wt) + str(pos+1) + str(mut)
            variant_list.append(variant)

    prism_df = pd.DataFrame({"variant" : variant_list, "score" : np.nan}, index = variant_list)
    return(prism_df)

def GEMME_to_prism(gemme_df, seq ,output_name , offset = 0):
    prism_df = generate_empty_prism_df(seq)
    gemme_df.index = aa_order_alphabetical
    for pos in gemme_df.columns:
        for aa in gemme_df.index:
            p = int(str(pos)[1:])
            wt = seq[p-1 + offset]
            mut = str(aa).upper()
            val = gemme_df.at[aa, pos]

            variant = str(wt) + str(p) + str(mut)
            #print(variant)
            prism_df.loc[variant, "score"] = val

    prism_df.reset_index(inplace = True, drop = True)
    prism_df = prism_df.dropna()
    prism_df.columns = ["variant", "gemme_score"]
    prism_df.to_csv(output_name,index=False,sep=' ')
    print(f'---> GEMME file converted')

def histogram_data(data_input,range_data,xlabel,title,where_save):
  fig,ax=plt.subplots(1,1,figsize=(6,4))
  ax.hist(data_input,range=range_data,bins=20,edgecolor='k',facecolor='#56b4e9')
  ax.set_xlabel(xlabel)
  ax.set_ylabel('counts')
  ax.set_title(title)

  plt.savefig(os.path.join(where_save,title+'_histogram.png'),dpi=300)

  plt.show()

def heatmap_classes_reordered(ext_score,label_cm,WT_mask,x,title_figure='',where_save='./',WT_cmap='gnuplot_r',nan_cmap='Dark2_r',figs=[30,7],xtick_spacing=10):
    mpl.rcParams['xtick.labelsize'] = 18
    mpl.rcParams['ytick.labelsize'] = 18
    mpl.rcParams['axes.labelsize']  = 20

    score=np.copy(ext_score)

    for i in range(score.shape[1]):
        #print(i,score[:,i][0:10])
        score[:,alphabetAA_D_D_reordered[i]]=ext_score[:,i]
        #print(alphabetAA_D_D_reordered[i],score[:,alphabetAA_D_D_reordered[i]][0:10])

    score_nan=np.copy(ext_score)
    score_nan[:]=np.inf
    
    for i in range(score.shape[0]):
        for j in range(score.shape[1]):
                if np.isnan(ext_score[i,j])==True and WT_mask[i]!=alphabetAA_D_L[j+1]:
                    
                    score_nan[i,alphabetAA_D_D_reordered[j]]=1
                    #print(WT_mask[i],alphabetAA_D_D_reordered[alphabetAA_D_L[j+1]])
            
    fig, ax = plt.subplots(figsize=(figs[0],figs[1]))
    ylabels=[ alphabetAA_L_D_reordered[i] for i in range(0,20)]
    if np.isnan(np.unique(score)).any():
      num_colors=len(np.unique(score))-1
    else:
      num_colors=len(np.unique(score))

    if num_colors==3:
      pos=ax.imshow(np.transpose(score), aspect = 'auto', cmap=mpl.colors.ListedColormap(["#009e74", "#56b4e9","#d55c00"]),interpolation='nearest')
    elif num_colors==4:
      pos=ax.imshow(np.transpose(score), aspect = 'auto', cmap=mpl.colors.ListedColormap(["#009e74", "#56b4e9","#d55c00","#f0e442"]),interpolation='nearest')
  
    #current_cmap = mpl.cm.get_cmap()
    current_cmap = plt.get_cmap()
    current_cmap.set_bad(color='gray')

    ax.set_yticks([i for i in range(0,20)])
    ax.set_yticklabels(ylabels)
    
    #ax.xaxis.set_ticks(np.arange(1, end, stepsize))

    plt.grid(axis='both',which='both',alpha=0.4)
    #plt.ylabel("mutation")
    plt.xlabel("residue")
    plt.ylim(-0.5,19.5)
    plt.xlim(x[0]-0.5,x[1]+0.5)
    start, end = ax.get_xlim()
    ax.xaxis.set_ticks(np.arange(start+0.5, end, xtick_spacing))
    ax.tick_params(axis='x',rotation=90)
    ax.tick_params(axis='y',rotation=90)
    ax.set_title(title_figure,fontsize=20)
    tmp=np.empty((len(WT_mask),20),dtype=float)
    tmp[:]=np.inf
    for i in range(0,len(WT_mask)):
        tmp[i,alphabetAA_D_D_reordered[alphabetAA_L_D[WT_mask[i]]-1]]=1
    
    plt.imshow(score_nan.T, cmap=nan_cmap, aspect = 'auto',interpolation='nearest')
    plt.imshow(tmp.T, cmap=WT_cmap, aspect = 'auto',interpolation='nearest')
    plt.savefig(os.path.join(where_save,title_figure+'_variant_map.png'),dpi=300)
    plt.show()
def residues_classes_reordered(ext_score,x,title_figure='',where_save='./',figs=[20,1],xtick_spacing=10):
  mpl.rcParams['xtick.labelsize'] = 18
  mpl.rcParams['ytick.labelsize'] = 18
  mpl.rcParams['axes.labelsize']  = 20
  fig,ax1 = plt.subplots(1,1,figsize=(figs[0],figs[1]))

  if 5. in ext_score:
    ax1.imshow(ext_score.reshape(1,-1),aspect= 'auto', cmap=mpl.colors.ListedColormap(["#009e74", "#56b4e9","#d55c00","#f0e442","#5A5A5A","#5A5A5A"]))
  else:
    if 3. in ext_score:
      ax1.imshow(ext_score.reshape(1,-1),aspect= 'auto', cmap=mpl.colors.ListedColormap(["#009e74", "#56b4e9","#d55c00","#f0e442"]))
    else:
      ax1.imshow(ext_score.reshape(1,-1),aspect= 'auto', cmap=mpl.colors.ListedColormap(["#009e74", "#56b4e9","#d55c00"]))

  ax1.set_yticks([])
  plt.xlim(x[0]-0.5,x[1]+0.5)

  start, end = ax1.get_xlim()
  ax1.xaxis.set_ticks(np.arange(start+0.5, end, xtick_spacing))
  ax1.tick_params(axis='x',rotation=90)

  ax1.set_xlabel('residue')
  ax1.set_title(title_figure,fontsize=20)
  plt.savefig(os.path.join(where_save,title_figure+'_residue_class.png'),dpi=300)
  plt.show()

In [14]:
#@title <b><font color='#56b4e9'>DATA PREPROCESSING:</font> Load sequence
#@markdown Input the query sequence and jobname for the target protein (it will be used for naming output files)\
#@markdown **N.B.: Re-rerunning the cell will delete previous runs and related uploaded input data.**
from google.colab import files
import os.path
import re

if 'input_path' in locals():
  shutil.rmtree(input_path)

query_sequence='SEQUENCE'#@param {type:"string"}

if ' ' in query_sequence:
  print('!!!! please check input sequence before proceeding: it may contains space characters !!!!')

jobname='test'#@param {type:"string"}

input_path = f"{jobname}_inputs"
if not os.path.exists(input_path):
  os.mkdir(input_path)


output_path = f"{jobname}_outputs"
if not os.path.exists(output_path):
  os.mkdir(output_path)

plot_folder=os.path.join(output_path,'figures')
if not os.path.exists(plot_folder):
  os.mkdir(plot_folder)


loaded_ddg=False
loaded_pdb=False
loaded_gemme=False
loaded_extra=False


In [ ]:
#@title <b><font color='#56b4e9'>DATA PREPROCESSING:</font> load $\Delta \Delta Gs$
#@markdown Run the cell to open the upload request and load $\Delta \Delta G$ mutagenesis file (in kcal/mol) -  the file format description is shown on the bottom of the notebook.\
#@markdown \
#@markdown **Select $\Delta \Delta G$ source:**
#@markdown - Rosetta $\Delta \Delta Gs$ (already correctly formatted)
#@markdown - RaSP $\Delta \Delta G$s  (use the output file: "prism_output_file", [RaSP webserver](https://colab.research.google.com/github/KULL-Centre/papers/blob/main/2022/ML-ddG-Blaabjerg-et-al/RaSPLab.ipynb))

ddG_file_format = "Rosetta" #@param ["Rosetta", "RaSP"]
#@markdown If the two sequences (query and input file) are different, the pipeline will align them and use only the matching position.

print_aligned_sequences = True #@param {type:"boolean"}
print('-> upload ddg file:')
uploaded_ddg = files.upload()

for fn in uploaded_ddg.keys():
    os.rename(fn, f"{jobname}_inputs/{fn}")
    
    if ddG_file_format == 'Rosetta':
      rosetta_scores=load_data(f"{jobname}_inputs/{fn}",query_sequence)
    elif ddG_file_format == 'RaSP':
      rosetta_scores=load_data(f"{jobname}_inputs/{fn}",query_sequence,column_score=2,print_sequences=print_aligned_sequences)
    print('--> rosetta ddg score loaded')

rosetta_scores=remove_WT_score(rosetta_scores,query_sequence)
rosetta_scores_norm=normalize_cutoff(rosetta_scores,0,5)
rosetta_scores_mean=np.nanmean(rosetta_scores_norm,axis=-1)

print('---> rosetta mean evaluated')

if ddG_file_format == 'Rosetta':
  histogram_data(rosetta_scores.flatten(),range_data=[0,5],xlabel='Rosetta $\Delta \Delta G$ [kcal/mol]',title=jobname+'_Rosetta',where_save=plot_folder)
elif ddG_file_format == 'RaSP':
  histogram_data(rosetta_scores.flatten(),range_data=[0,5],xlabel='RaSP $\Delta \Delta G$ [kcal/mol]',title=jobname+'_RaSP',where_save=plot_folder)

loaded_ddg = True


In [ ]:
#@title <b><font color='#56b4e9'>DATA PREPROCESSING:</font> load PDB file or AF2 predicted structure
#@markdown Run the cell to open the upload request and load the structure of target protein in PDB format.

#@markdown **Select input chain:**
 
pdb_chain='0'#@param ["0",'1',"2",'3',"4",'5',"6",'7','8','9']
#@markdown If the two sequences (query and input file) are different, the pipeline will align them and use only the matching position.
print_aligned_sequences = True #@param {type:"boolean"}



print('-> upload pdb file:')
pdb_path = f"{jobname}_inputs"
uploaded_pdb = files.upload()

for fn in uploaded_pdb.keys():
  os.rename(fn, f"{jobname}_inputs/{fn}")

  print('--> pdb loaded')
  print()
  wcn_scores=WCN(f"{jobname}_inputs/{fn}",'ca',query_sequence,chain=int(pdb_chain),print_sequences=print_aligned_sequences)
  print('---> wcn evaluated')
  loaded_pdb = True

histogram_data(wcn_scores.flatten(),range_data=[0,20],xlabel='Weighted Contact Number',title=jobname+'_WCN',where_save=plot_folder)


In [ ]:
#@title <b><font color='#56b4e9'>DATA PREPROCESSING:</font> load GEMME file


print('-> upload GEMME file:')
gemme_path = f"{jobname}_inputs"
uploaded_gemme = files.upload()
#@markdown Run the cell to open the upload request and load $\Delta \Delta E$ mutagenesis file  -  the file format description is shown on the bottom of the notebook.\
#@markdown \
#@markdown **Select GEMME file format:**
#@markdown - Pipeline format (already formatted, two columns:  [W2M, score], see github for complete example)
#@markdown - GEMME webserver format ("normPred_evolCombi.txt", [webserver](http://www.lcqb.upmc.fr/GEMME/submit.html)). **IF** you are using this option the query and the file sequences <b><font color='#D55C00'>HAVE TO MATCH</font>.


GEMME_file_format = "Pipeline format" #@param ["Pipeline format", "GEMME webserver format"]

#@markdown If the two sequences (query and input file) are different, the pipeline will align them and use only the matching position.
print_aligned_sequences = True #@param {type:"boolean"}
import mdtraj as md

for fn in uploaded_gemme.keys():
    os.rename(fn, f"{jobname}_inputs/{fn}")
    print('--> GEMME file found')

    if GEMME_file_format == "GEMME webserver format":
      input_gemme=pd.read_csv(f"{jobname}_inputs/{fn}",delim_whitespace=True)
      GEMME_to_prism(input_gemme,query_sequence,f"{jobname}_inputs/reformatted_{fn}",0)
      gemme_scores=load_data(f"{jobname}_inputs/reformatted_{fn}",query_sequence,print_sequences=print_aligned_sequences)
    else:
      gemme_scores=load_data(f"{jobname}_inputs/{fn}",query_sequence,print_sequences=print_aligned_sequences)
  
      print('---> GEMME scores loaded')

gemme_scores=remove_WT_score(gemme_scores,query_sequence)
gemme_scores_mean=np.nanmean(gemme_scores,axis=-1)

print('---> GEMME mean score evaluated')
histogram_data(gemme_scores.flatten(),range_data=[-7,0],xlabel='GEMME $\Delta \Delta E$',title=jobname+'_GEMME',where_save=plot_folder)
loaded_gemme = True

In [9]:
#@title <b><font color='#56b4e9'>DATA PREPROCESSING:</font> Evaluating extra necessary features
#@markdown Run the cell to evaluate the extra necessary features, including:
#@markdown - $\Delta \Delta G$ and conservation neighbor scores
#@markdown - Weighted contact number

rosetta_neigbour_scores=neighbor_scores(rosetta_scores_mean,1)

gemme_neigbour_scores=neighbor_scores(gemme_scores_mean,1)


hydrophobicity_mut=np.empty((len(query_sequence),20),dtype=float)
hydrophobicity_mut[:]=np.nan

for i in range(len(query_sequence)):
    for j in range(20):
        hydrophobicity_mut[i,j]=AA_to_hydrophobicity_scores[alphabetAA_D_L[j+1]]

loaded_extra=True

In [ ]:
#@title <b><font color='#56b4e9'>MODEL RUN:</font> Variant prediction and residue classification
#@markdown Run the cell to use the trained catboost model and generate predictions for the query protein at variant and residue level.
from catboost import CatBoostClassifier
from glob import glob
if ((loaded_ddg) and (loaded_gemme) and (loaded_pdb) and (loaded_extra)):
  cat=CatBoostClassifier()
  X,map=features_validation([gemme_scores,rosetta_scores, gemme_scores_mean,rosetta_scores_mean,hydrophobicity_mut,gemme_neigbour_scores,rosetta_neigbour_scores,wcn_scores],query_sequence)
  if ddG_file_format == 'Rosetta':
    if os.path.exists('./cat_trained_24jun22.cbm'):
      cat.load_model("./cat_trained_24jun22.cbm")
    else:
      print(f'Error: missing/incorrect model')
  else:
    if os.path.exists('./cat_cavity_trained_21march23.cbm'):
        cat.load_model("./cat_cavity_trained_21march23.cbm")
    else:
      print(f'Error: missing/incorrect model')

  print(f"-> Model loaded")

  prediction=cat.predict(X)

  print(f"--> Prediction succeed ")

  variant_pred=np.empty((len(query_sequence),20),dtype=float)
  variant_pred[:]=np.nan

  for i,(n,m) in enumerate(zip(map,prediction)):
    variant_pred[n[0],n[1]]=m

  print(f"---> Variant score evaluated")

  residue_mode_pred=retrieve_residue_label_pred(prediction,map,query_sequence,0.5)

  print(f"----> Residue score evaluated")

else:
  if loaded_ddg!=True:
    print(f'Error: missing ddGs')
  if loaded_gemme!=True:
    print(f'Error: missing GEMME scores')
  if loaded_pdb!=True:
      print(f'Error: missing pdb and wcn scores')
  if loaded_extra!=True:
      print(f'Error: missing additional feature scores')

In [ ]:
#@title <b><font color='#56b4e9'>Show results: </font>
#@markdown Plot and save the results of the classification for variants and residues of the query sequence\
#@markdown \
#@markdown **Color keys:**
#@markdown - <b><font color='#009E74'>GREEN </font></b>: WT-like (variant and residue)
#@markdown - <b><font color='#56b4e9'>BLUE </font></b>: Stable but inactive (variant) or **functional position** (residue)
#@markdown - <b><font color='#D55C00'>RED </font></b>: Total-loss (variant and residue)
#@markdown - <b><font color='#F0E442'>YELLOW </font></b>: Active but unstable (variant)
#@markdown - <b>GRAY</b>: Class not assigned (residue)

#@markdown \
#@markdown **N.B.:** residue numeration in the plots starts from 0.

heatmap_classes_reordered(variant_pred,'\u0394\u0394G [a.u]',query_sequence,[0,len(query_sequence)-1],WT_cmap='coolwarm',xtick_spacing=20,figs=[40,6],where_save=plot_folder,title_figure=jobname)

residues_classes_reordered(residue_mode_pred,[0,len(query_sequence)-1],title_figure=jobname,where_save=plot_folder,figs=[40,1],xtick_spacing=50)

In [ ]:
#@title <b><font color='#56b4e9'>DOWNLOAD RESULTS: </font>

mutation_list=[]
residue_list=[]
for i in range(0,len(query_sequence)):
  residue_list.append(query_sequence[i]+str(i+1))
  for j in range(20):
    mutation_list.append(query_sequence[i]+str(i+1)+alphabetAA_D_L[j+1])


print_feature_df=pd.DataFrame({'Mutation': np.array(mutation_list).flatten(),
                        'GEMME variant' : gemme_scores.flatten(),'GEMME mean residue' : np.tile(gemme_scores_mean,(20,1)).T.flatten(), 'GEMME neighbours' : np.tile(gemme_neigbour_scores,(20,1)).T.flatten(),
                        'Rosetta ddG variant' : rosetta_scores_norm.flatten() ,'Rosetta ddG mean residue' :  np.tile(rosetta_scores_mean,(20,1)).T.flatten(),'Rosetta ddG neigbours' : np.tile(rosetta_neigbour_scores,(20,1)).T.flatten(),
                       'Hydrophobicity variant' : hydrophobicity_mut.flatten(), 'WCN residue' :  np.tile(wcn_scores,(20,1)).T.flatten()})

print_variant_df=pd.DataFrame({'Mutation': np.array(mutation_list).flatten(),
                        'Variant class ' : np.vectorize(alphabet_class_variant_d_l.get)(variant_pred.flatten()),'Variant class (digit)' : variant_pred.flatten()})

print_residue_df=pd.DataFrame({'Residue': np.array(residue_list).flatten(),
                        'Residue class ' : np.vectorize(alphabet_class_residue_d_l.get)(residue_mode_pred.flatten()),'Residue class (digit)' : residue_mode_pred.flatten(),
                        '# WT-like variant' : count_class_variant_position(variant_pred,0),'# SBI variant' : count_class_variant_position(variant_pred,1),'# Total-loss variant' : count_class_variant_position(variant_pred,2)})

## print files:
#@markdown Select the output format file:
output_format = "csv" #@param ["csv","excel"]
#@markdown Mark the next box if you want a zip file with results (if not file will be store in the colab output folder)
download_results = True #@param {type:"boolean"}
if output_format== 'csv':
  print_feature_df.to_csv(f"{jobname}_outputs/"+f"{jobname}_variant_features.csv",sep=',')
  print_variant_df.to_csv(f"{jobname}_outputs/"+f"{jobname}_variant_predictions.csv",sep=',')
  print_residue_df.to_csv(f"{jobname}_outputs/"+f"{jobname}_residue_predictions.csv",sep=',')

if output_format== 'excel':
  print_feature_df.to_excel(f"{jobname}_outputs/"+f"{jobname}_variant_features.xlsx")
  print_variant_df.to_excel(f"{jobname}_outputs/"+f"{jobname}_variant_predictions.xlsx")
  print_residue_df.to_excel(f"{jobname}_outputs/"+f"{jobname}_residue_predictions.xlsx")
  
  
if download_results:
  os.system( "zip -r {} {}".format( f"{jobname}_outputs.zip" , f"{jobname}_outputs" ) )
  files.download(f"{jobname}_outputs.zip")


<b><font color='#56b4e9'>EXTRA </font></b>

\\

**File Format**

Input files containing the conservation scores or thremodynamic stability changes should be properly formatted.

\\

>EXAMPLE:

>For a target protein with 45 residues, the scores file should be formatted like this: 

>Mutation  Score

>M1A       2.4  
M1C       1.2  
..  
M1=
M1W       1.3  
C2A       0.2   
..  
..  
Y45W       0.3
  
>N.B.: Synonymous mutations should be skipped or reported as mutation to =. Stop mutations has to been removed from the list.

\\

**Known problems:**

- Residues with numeration index below 0 are not supported by the output file parser and thus they deleted from the pdb in the pre-processing step.
- Insertions, deletions or missenses in any of the input file sequences (scores and PDB) compared to the query sequence are resolved using the Biopython alignment function. If the difference between the input sequences is too large, this may lead to incorrect alignment.

\\

**License:**

The source code and model's parameters are licensed under the permissive Apache Licence, Version 2.0.
 Additionally, this notebook uses the reduce source code which license could be find in `/content/src/pdb_parser_scripts/reduce/`

\\

**Bugs:**

For any bugs please report the issue on the project [Github](https://github.com/KULL-Centre/papers/tree/main/2022/functional-sites-cagiada-et-al) or contact one of the listed authors in the connected [manuscript](https://www.biorxiv.org/content/10.1101/2022.07.14.500015v1.full).

\\

**Citing this work:**

If you use our model please cite:

Cagiada, M., Bottaro, S., Lindemose, S., Schenstrøm, S. M., Stein, A., Hartmann-Petersen, R., & Lindorff-Larsen, K. (2022). Discovering functionally important sites in proteins. bioRxiv, 2022-07.

```
@article{cagiada2022discovering,
  title={Discovering functionally important sites in proteins},
  author={Cagiada, Matteo and Bottaro, Sandro and Lindemose, S{\o}ren and Schenstr{\o}m, Signe M and Stein, Amelie and Hartmann-Petersen, Rasmus and Lindorff-Larsen, Kresten},
  journal={bioRxiv},
  pages={2022--07},
  year={2022},
  publisher={Cold Spring Harbor Laboratory}
}
```
